In [1]:
import transformers

In [2]:
from transformers import AutoModelForMaskedLM, AutoTokenizer
import torch
import numpy as np

model_name = 'cointegrated/rubert-tiny'
model = AutoModelForMaskedLM.from_pretrained(model_name)
tokenizer = AutoTokenizer.from_pretrained(model_name)

def score(model, tokenizer, sentence):
    tensor_input = tokenizer.encode(sentence, return_tensors='pt')
    repeat_input = tensor_input.repeat(tensor_input.size(-1)-2, 1)
    mask = torch.ones(tensor_input.size(-1) - 1).diag(1)[:-2]
    masked_input = repeat_input.masked_fill(mask == 1, tokenizer.mask_token_id)
    labels = repeat_input.masked_fill( masked_input != tokenizer.mask_token_id, -100)
    with torch.inference_mode():
        loss = model(masked_input, labels=labels).loss
    return np.exp(loss.item())

print(score(sentence='London is the capital of Great Britain.', model=model, tokenizer=tokenizer)) 
# 4.541251105675365
print(score(sentence='''o i guess
to that killd space in a
motion you if look
should waterdrops
good by wonderd yielded not out thee henrys and was quit on
such when properly dear to judgment ignorant answer

coriolanus
hence it richard another shed will me neglected of on a for infected thank hour theres hast like with art him royalize tis which in mine yet
let proud
can love

romeo
out

benvolio
of in
and two shame stony with twas after and shall you
that with citizen
why gallant courtesy
but lesson saw it ambassador
but a you respected boon
and i i and him had any i right
if begun ends lost sighs was was drownd that him to here been friend thee you may full once no resident my with flayd murderer
where can i a my love
with his weepst knowledge can i a simple

benvolio
by be kill richard mourn in i denied would live
canst cannot one to him pair
that knockd edward order being thee
but fierce of i take the you
commend like his the fathers so blush touch in king toward but therefore sir must by one slight farewell give not my it duke thine side
i the banks
if he york
and stands
against court
and then for sense
''', model=model, tokenizer=tokenizer)) 
# 6.162017238332462